In [7]:
import requests
import json
import time
import csv

In [5]:
csv_file = "UPRN Mapping Document v2/OPAS-Table 1.csv"
mapping = dict()
with open(csv_file, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    counter = 0
    for row in spamreader:
        counter += 1
        if counter == 1:
            continue
        title = row[0]
        emu = row[-1]
        if emu != "":
            title = title.split(" Lift/")[0]
            title = title.replace("Stn", "Station")
            mapping[emu] = title
            # print(mapping[emu])

FileNotFoundError: [Errno 2] No such file or directory: 'UPRN Mapping Document v2/OPAS-Table 1.csv'

In [8]:
# url = "https://europe-west1-infrahack.cloudfunctions.net/lifts-tfl"
url = "https://europe-west1-infrahack.cloudfunctions.net/lifts-nr"

re = requests.get(url, params={"timestamp": "auto",
                               "time_speedup": 10,
                               "events_lookback_period_sec": 166400})

print(f"{re.status_code} returned")

200 returned


In [9]:
re.json()['data']

[{'lift_id': '1085', 'currently_operational': True, 'events': []},
 {'lift_id': '1086', 'currently_operational': True, 'events': []},
 {'lift_id': '1088', 'currently_operational': True, 'events': []},
 {'lift_id': '8159',
  'currently_operational': True,
  'events': [{'event_code': 'Call Failed',
    'event_type': 'Event',
    'event_condition': 'OK',
    'occurred_lift_position': '1',
    'restored_lift_position': '1',
    'start': 1543720350.0,
    'end': 1543720514.0},
   {'event_code': 'Lift on Car Preference',
    'event_type': 'Event',
    'event_condition': 'Not in Service',
    'occurred_lift_position': '1',
    'restored_lift_position': '1',
    'start': 1543708606.0,
    'end': 1543714946.0},
   {'event_code': 'Service Not Available.',
    'event_type': 'Event',
    'event_condition': 'Not in Service',
    'occurred_lift_position': '1',
    'restored_lift_position': '1',
    'start': 1543708606.0,
    'end': 1543720248.0},
   {'event_code': 'Lift on Car Preference',
    'even

In [58]:
print(len(re.json()['data']))
with open('api_result.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["lift_id", "operational", "event"] + [f"event{x}" for x in range(1, 100)])
    for lift in re.json()['data']:
        print(lift)

        spamwriter.writerow([lift.get("lift_id"), lift.get("currently_operational")] + [x.get("event_code") for x in lift.get("events")])




16
{'lift_id': '1085', 'currently_operational': True, 'events': []}
{'lift_id': '1086', 'currently_operational': True, 'events': []}
{'lift_id': '1088', 'currently_operational': True, 'events': []}
{'lift_id': '8159', 'currently_operational': True, 'events': [{'event_code': 'Lift on Car Preference', 'event_type': 'Event', 'event_condition': 'Not in Service', 'occurred_lift_position': '1', 'restored_lift_position': '1', 'start': 1542584478.0, 'end': 1542593384.0}, {'event_code': 'Service Not Available.', 'event_type': 'Event', 'event_condition': 'Not in Service', 'occurred_lift_position': '1', 'restored_lift_position': '1', 'start': 1542584478.0, 'end': 1542593426.0}, {'event_code': 'Pre-lock Failure Occurred', 'event_type': 'Event', 'event_condition': 'OK', 'occurred_lift_position': '1', 'restored_lift_position': '1', 'start': 1542535078.0, 'end': 1542535148.0}, {'event_code': 'Call Failed', 'event_type': 'Event', 'event_condition': 'OK', 'occurred_lift_position': '1', 'restored_lift_p

In [17]:
filtered = [x for x in data['data'] if x['currently_operational'] == False]

In [25]:
filtered

[{'lift_id': '8162',
  'currently_operational': False,
  'events': [{'event_code': 'Lost Main Controller Supply',
    'event_type': 'Event',
    'event_condition': 'Not Operational',
    'occurred_lift_position': '3',
    'restored_lift_position': None,
    'start': 1556578704.0,
    'end': None},
   {'event_code': 'Not Operational',
    'event_type': 'Out of Service',
    'event_condition': 'Not Operational',
    'occurred_lift_position': '3',
    'restored_lift_position': None,
    'start': 1556578704.0,
    'end': None},
   {'event_code': 'Service Not Available.',
    'event_type': 'Event',
    'event_condition': 'Not in Service',
    'occurred_lift_position': '3',
    'restored_lift_position': None,
    'start': 1556578698.0,
    'end': None}]},
 {'lift_id': '8174',
  'currently_operational': False,
  'events': [{'event_code': 'Lost Main Controller Supply',
    'event_type': 'Event',
    'event_condition': 'Not Operational',
    'occurred_lift_position': '1',
    'restored_lift_pos

In [7]:
_id = filtered[0]['lift_id']
print(mapping[_id])

London Kings Cross Station


In [8]:
import pymongo
db = pymongo.MongoClient().infrahack

In [9]:
db.stations.insert_many([{"_id": emu_id,
                      "station_name": name} for (emu_id, name) in mapping.items()])

In [36]:
session = requests.Session()

re = session.post("https://nwridsrv.tvc-cmsanywhere.co.uk/Account/Login?returnurl=%2Fconnect%2Fauthorize%2Fcallback%3Fclient_id%3DCMSAnywhere%26redirect_uri%3Dhttps%253A%252F%252Fnwr.tvc-cmsanywhere.co.uk%252Fsignin-oidc%26response_type%3Dcode%2520id_token%26scope%3Dopenid%2520profile%2520MonitoringInternalAPI%2520MonitoringAPI%2520MonitoringDialOutAPI%2520MonitoringAdminAPI%2520offline_access%2520roles%26response_mode%3Dform_post%26nonce%3D636937309931477745.ODAwMmYyMDEtNGU1ZC00Yzc0LTkzYTUtODNiOTYyZTJkY2VmNWQ4ZTAzMGItMzM3MC00ZmIyLTlmYjctM2FlYzBkNThmNTU2%26state%3DCfDJ8A2fKo5tMgBInkAsJyKK0Pp8-POlphkAPn01MlswJJ2rBxrMa5851iXINB6g1-pia7SupW3aSlnheBl6v3RU5jVofhJkyy-aqX_jLWktVxj4mQi4jzxxTU7N7vr45uZbfPMSa5JtyllZ1qHH3qrTFoO3wQBHwayRZwgp4Xa0jiMqiGK7aVTYpIFCyzyOFNkEHvmxZTdN8uKftFtAttijymWFIEGpWde_jPbL4r3kaIHVHLLAONXqoZG43ZRBwjyBnpENYbu2tse_XJTGBA2oIIUSJCCD_HJ3dJQSWEQy3nAyh-GKyfqgdtc5nFAwY78K9wcCxnKXmR03d_wLbotIYXklxFcgJ6Mgq6i-1G3cahsG%26x-client-SKU%3DID_NETSTANDARD2_0%26x-client-ver%3D5.3.0.0",
                  data={"Username": "hacktrain@NWR", "Password": "Hacktrain!123"})
print(re.status_code)

url = "https://nwrapi.tvc-cmsanywhere.co.uk/api/emu/currentavailability?_=1558133967292"
re = session.get(url, auth = ("hacktrain@NWR", "Hacktrain!123"))
print(re.status_code)
re.text

400
401


''

In [1]:

import sys
sys.path.append("../")
from lib.helper import *
from lib.db import MongoClient
c = MongoClient("mongodb://localhost:27017")

In [2]:
print(get_faults())

>> getting faults
{'8162': 1556578704.0, '8174': 1551973202.0, '8177': 1552091564.0, '8178': 1555217720.0, '8204': 1552214106.0, '8540': 1553727066.0}


In [3]:
print(load_cms_faults())

{'5201': 1557983400.0, '7046': 1558141440.0, '5211': 1558126260.0, '5208': 1557938280.0, '6105': 1558150680.0, '6117': 1557761160.0, '6135': 1557909360.0, '6134': 1558132260.0, '8157': 1558149360.0, '8189': 1558130160.0, '4030': 1558144800.0, '6110': 1557904980.0, '35': 1557392760.0, '7044': 1558137420.0, '7042': 1558136220.0, '7043': 1558135020.0, '8232': 1558139880.0, '8233': 1558141380.0, '8234': 1558083660.0, '8383': 1557071640.0, '7002': 1558001520.0, '6114': 1557219480.0, '8241': 1558138020.0, '2064': 1558129020.0, '7518': 1558136160.0, '2049': 1558150620.0, '2006': 1558048740.0, '8556': 1557813720.0, '8118': 1558147320.0, '8113': 1558119360.0, '8112': 1558119180.0, '8111': 1558128840.0, '2056': 1558151520.0, '3045': 1558014540.0, '5036': 1558120140.0, '5035': 1557262680.0, '9902': 1557936060.0, '8175': 1558151040.0, '1065': 1558110360.0, '1185': 1558079040.0, '1186': 1558147320.0, '9892': 1557751140.0, '9869': 1557930060.0, '6138': 1558139160.0, '9912': 1558101540.0, '9908': 155

In [4]:
print(prepare_faults(c))

>> getting faults
>> {'8162': 1556578704.0, '8174': 1551973202.0, '8177': 1552091564.0, '8178': 1555217720.0, '8204': 1552214106.0, '8540': 1553727066.0}
>> {'8162': 1556578704.0, '8174': 1551973202.0, '8177': 1552091564.0, '8178': 1555217720.0, '8204': 1552214106.0, '8540': 1553727066.0, '5201': 1557983400.0, '7046': 1558141440.0, '5211': 1558126260.0, '5208': 1557938280.0, '6105': 1558150680.0, '6117': 1557761160.0, '6135': 1557909360.0, '6134': 1558132260.0, '8157': 1558149360.0, '8189': 1558130160.0, '4030': 1558144800.0, '6110': 1557904980.0, '35': 1557392760.0, '7044': 1558137420.0, '7042': 1558136220.0, '7043': 1558135020.0, '8232': 1558139880.0, '8233': 1558141380.0, '8234': 1558083660.0, '8383': 1557071640.0, '7002': 1558001520.0, '6114': 1557219480.0, '8241': 1558138020.0, '2064': 1558129020.0, '7518': 1558136160.0, '2049': 1558150620.0, '2006': 1558048740.0, '8556': 1557813720.0, '8118': 1558147320.0, '8113': 1558119360.0, '8112': 1558119180.0, '8111': 1558128840.0, '2056': 